In [1]:
#from binarytree import Node
import random

In [2]:
def _build_tree_string(root, curr_index, index=False, delimiter='-'):
    if root is None or root.value is None:
        return [], 0, 0, 0
    
    line1 = []
    line2 = []
    if index:
        node_repr = '{}{}{}'.format(curr_index, delimiter, root.value)
    else:
        node_repr = str(root.value) + "(" + root.color + ")"

    new_root_width = gap_size = len(node_repr)

    # Get the left and right sub-boxes, their widths, and root repr positions
    l_box, l_box_width, l_root_start, l_root_end = \
        _build_tree_string(root.left, 2 * curr_index + 1, index, delimiter)
    r_box, r_box_width, r_root_start, r_root_end = \
        _build_tree_string(root.right, 2 * curr_index + 2, index, delimiter)

    # Draw the branch connecting the current root node to the left sub-box
    # Pad the line with whitespaces where necessary
    if l_box_width > 0:
        l_root = (l_root_start + l_root_end) // 2 + 1
        line1.append(' ' * (l_root + 1))
        line1.append('_' * (l_box_width - l_root))
        line2.append(' ' * l_root + '/')
        line2.append(' ' * (l_box_width - l_root))
        new_root_start = l_box_width + 1
        gap_size += 1
    else:
        new_root_start = 0

    # Draw the representation of the current root node
    line1.append(node_repr)
    line2.append(' ' * new_root_width)

    # Draw the branch connecting the current root node to the right sub-box
    # Pad the line with whitespaces where necessary
    if r_box_width > 0:
        r_root = (r_root_start + r_root_end) // 2
        line1.append('_' * r_root)
        line1.append(' ' * (r_box_width - r_root + 1))
        line2.append(' ' * r_root + '\\')
        line2.append(' ' * (r_box_width - r_root))
        gap_size += 1
    new_root_end = new_root_start + new_root_width - 1

    # Combine the left and right sub-boxes with the branches drawn above
    gap = ' ' * gap_size
    new_box = [''.join(line1), ''.join(line2)]
    for i in range(max(len(l_box), len(r_box))):
        l_line = l_box[i] if i < len(l_box) else ' ' * l_box_width
        r_line = r_box[i] if i < len(r_box) else ' ' * r_box_width
        new_box.append(l_line + gap + r_line)

    # Return the new box, its width and its root repr positions
    return new_box, len(new_box[0]), new_root_start, new_root_end

In [3]:
class RBNode(object):
    def __init__(self, value=None, color='r'):
        self.value = value
        self.left = None
        self.right = None
        self.parent = None
        self.color = color
                
    def __str__(self):
        lines = _build_tree_string(self, 0, False, '-')[0]
        return '\n' + '\n'.join((line.rstrip() for line in lines))

In [4]:
root = RBNode()
#root.left = RBNode(2)
#root.right = RBNode(3)
#root.left.left = RBNode(4)

In [5]:
print(root)

In [72]:
class RBTree:
    def __init__(self, value=None):
        # guard node
        self.T = RBNode(None, 'b')
        self.root = self.T
        
        if value:
            self.insert(value)
    
    def insert(self, value):
        x, y = self.root, self.root
        z = RBNode(value)
        
        while x != self.T:
            y = x
            if value < x.value:
                x = x.left
            else:
                x = x.right
        if y == self.T:
            self.root = z
        elif y.value > value:
            y.left = z
        else:
            y.right = z
        
        z.left = self.T
        z.right = self.T
        z.parent = y
        
        self.T.parent = z
        
        self.insert_fixup(z)
    
    def insert_fixup(self, z):
        #import pdb; pdb.set_trace()
        while z.parent.color == 'r':
            if z.parent == z.parent.parent.left:
                y = z.parent.parent.right
                if y.color == 'r':
                    z.parent.color = 'b'
                    y.color = 'b'
                    z.parent.parent.color = 'r'
                    z = z.parent.parent
                elif z == z.parent.right:
                    z = z.parent
                    self.left_rotate(z)
                else:
                    z.parent.color = 'b'
                    z.parent.parent.color = 'r'
                    self.right_rotate(z.parent.parent)
            else:
                y = z.parent.parent.left
                if y.color == 'r':
                    z.parent.color = 'b'
                    y.color = 'b'
                    z.parent.parent.color = 'r'
                    z = z.parent.parent
                elif z == z.parent.left:
                    z = z.parent
                    self.right_rotate(z)
                else:
                    z.parent.color = 'b'
                    z.parent.parent.color = 'r'
                    self.left_rotate(z.parent.parent)
        
        self.root.color = 'b'
        self.T.color = 'b'
    
    def left_rotate(self, x):
        y = x.right
        
        # no y
        if y == self.T or x == self.T:
            return
        
        # put y.left to x.right
        x.right = y.left
        
        # restruct y.left's parent
        if y.left != self.T:
            y.left.parent = x
        
        # restruct y's parent
        y.parent = x.parent
        # x is root
        if x.parent == self.T:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
            
        # put x to y.left
        y.left = x
        x.parent = y
        
    def right_rotate(self, x):
        y = x.left
        
        # no y
        if y == self.T or x == self.T:
            return
        
        # put y.right to x.left
        x.left = y.right
        
        # restruct y.left's parent
        if y.right != self.T:
            y.right.parent = x
        
        # restruct y's parent
        y.parent = x.parent
        # x is root
        if x.parent == self.T:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
            
        # put x to y.right
        y.right = x
        x.parent = y
    
    '''
    1. all node's color is red or black
    2. root node's color is black
    3. all leaf(T)'c color is black
    4. if a node is red, its children is black
    5. path from root to leaf, contail same count black nodes
    '''
    def is_rbtree(self):
        
        # rule 2
        if self.root.color != 'b':
            return False
        
        # rule 3
        if self.T.color != 'b':
            return False

        # rule 4, 5
        def check_rbtree_rule(root):
            # dfs
            if root == self.T:
                return True, 0
            
            if root.color == 'r' and (root.left.color != 'b' 
                or root.right.color != 'b'):
                return False, -1
            
            ans1, l_count = check_rbtree_rule(root.left)
            ans2, r_count = check_rbtree_rule(root.right)
            
            if not ans1 or not ans2:
                return False, -2
            
            if l_count != r_count:
                return False, -2
            else:
                if root.color == 'b':
                    l_count += 1
                return True, l_count
            
        return check_rbtree_rule(self.root)[0]

In [85]:
def test():
    nums = random.sample(range(50), 20)
    #nums = [3, 5, 9, 6, 19, 1, 18, 11, 13]
    print(nums)
    rbtree = RBTree()
    for n in nums:
        rbtree.insert(n)
        #print(rbtree.root)
    
    print(rbtree.root)
    
    return rbtree

In [86]:
a = test()

[31, 36, 16, 34, 10, 35, 19, 49, 1, 44, 45, 38, 17, 26, 47, 46, 33, 24, 9, 6]

                         __________________________31(b)________________________________
                        /                                                               \
             ________16(b)________                                       ______________44(b)______________
            /                     \                                     /                                 \
   _______9(r)__               __19(r)________                     __35(r)__                     ________47(r)__
  /             \             /               \                   /         \                   /               \
1(b)_          10(b)       17(b)           __26(b)           __34(b)       36(b)__           45(b)__           49(b)
     \                                    /                 /                     \                 \
     6(r)                              24(r)             33(r)                   

In [87]:
a.is_rbtree()

True